[Getting Started Developing with Python and DynamoDB](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.html)


- start  Local DynamoDB

```
$ cat start_dynamodb 
java -Djava.library.path=~/projects/aws-cert/dynamodb/DynamoDBLocal_lib -jar ~/projects/aws-cert/dynamodb/DynamoDBLocal.jar -sharedDb

export DYNAMO_ENDPOINT="http://localhost:8000"

```

- [A GUI for Local DynamoDB— dynamodb-admin](https://medium.com/swlh/a-gui-for-local-dynamodb-dynamodb-admin-b16998323f8e)

```
$ sudo npm install -g npm
$ dynamodb-admin

dynamodb-admin listening on http://localhost:8001 
```

- Documentation
    - https://www.fernandomc.com/posts/ten-examples-of-getting-data-from-dynamodb-with-python-and-boto3/


In [9]:
from boto3.dynamodb.conditions import Key
from botocore.exceptions import ClientError

### [Step 1: Create a Table](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.01.html)



In [10]:
DYNAMO_ENDPOINT = "http://localhost:8000"

In [11]:
import boto3
# Get the service resource.
dynamodb = boto3.resource('dynamodb', endpoint_url=DYNAMO_ENDPOINT)

In [12]:
def create_movie_table(table_name='Movies', dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url=DYNAMO_ENDPOINT)
        
    try:
        table = dynamodb.Table(table_name)
        status = table.table_status
    except:

        # Error thrown when creating a pre-existing table
        # ResourceInUseException: An error occurred (ResourceInUseException) when calling the CreateTable operation: 
        # Cannot create preexisting table
        table = dynamodb.create_table(
            TableName=table_name,
            KeySchema=[
                {
                    'AttributeName': 'year',
                    'KeyType': 'HASH'  # Partition key
                },
                {
                    'AttributeName': 'title',
                    'KeyType': 'RANGE'  # Sort key
                }
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'year',
                    'AttributeType': 'N'
                },
                {
                    'AttributeName': 'title',
                    'AttributeType': 'S'
                },

            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        )
    return table

In [13]:
table = create_movie_table(table_name="Movies", dynamodb=dynamodb)
print("Table status:", table.table_status)

Table status: ACTIVE


### [Step 2: Load Sample Data](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.02.html)

In [31]:
from decimal import Decimal
import json

def load_movies(movies, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Movies')
    with table.batch_writer() as batch:
        for movie in movies:
            year = int(movie['year'])
            title = movie['title']
            print("Adding movie:", year, title)
            batch.put_item(Item=movie)
        
with open("moviedata.json") as json_file:
    movie_list = json.load(json_file, parse_float=Decimal)
load_movies(movie_list)

### [Step 3: CRUD - Create, Read, Update, and Delete an Item](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.03.html)

given (pk,sk), one can create/update/delete/get item for that primary_key (pk,sk)

#### Step 3.1: Create a New Item  - put_item()

In [14]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if abs(o) % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

# dynamodb = boto3.resource('dynamodb', region_name='us-west-2', endpoint_url="http://localhost:8000")

table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

response = table.put_item(
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':"Nothing happens at all.",
            'rating': decimal.Decimal(0)
        }
    }
)

print("put_item succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))


put_item succeeded:
{
    "ResponseMetadata": {
        "RequestId": "60f3bb40-3438-4dea-899b-0529fa157428",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "x-amz-crc32": "2745614147",
            "x-amzn-requestid": "60f3bb40-3438-4dea-899b-0529fa157428",
            "content-length": "2",
            "server": "Jetty(8.1.12.v20130726)"
        },
        "RetryAttempts": 0
    }
}


creating the same New Item will not cause duplicate

In [17]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if abs(o) % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

# dynamodb = boto3.resource('dynamodb', region_name='us-west-2', endpoint_url="http://localhost:8000")

table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

response = table.put_item(
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':"Nothing happens at all.",
            'rating': decimal.Decimal(-10)
        }
    }
)

print("put_item succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))


put_item succeeded:
{
    "ResponseMetadata": {
        "RequestId": "94fc1100-6783-41ca-89d4-ec62da03c45a",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "content-type": "application/x-amz-json-1.0",
            "x-amz-crc32": "2745614147",
            "x-amzn-requestid": "94fc1100-6783-41ca-89d4-ec62da03c45a",
            "content-length": "2",
            "server": "Jetty(8.1.12.v20130726)"
        },
        "RetryAttempts": 0
    }
}


#### Step 3.2: Read an Item - get_item()

In [7]:
table = dynamodb.Table('Movies')

In [8]:
table

dynamodb.Table(name='Movies')

In [6]:
title = "The Big New Movie"
year = 2015

try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    print("get_item succeeded:")
    print(json.dumps(item, indent=4, cls=DecimalEncoder))

Cannot do operations on a non-existent table


#### Step 3.3: Update an Item

`put_item()`  does `update_item()`

In [19]:
table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015
rating = 5.5
plot = "Everything happens all at once."
actors = ["Larry", "Moe", "Curly"]

try:
    response = table.update_item(
        Key={
            'year': year,
            'title': title
        },
        UpdateExpression="set info.rating=:r, info.plot=:p, info.actors=:a",
        ExpressionAttributeValues={
            ':r': decimal.Decimal(rating),
            ':p': plot,
            ':a': actors
        },
        ReturnValues="UPDATED_NEW"
    )
except ClientError as e:
    print(e.response['Error']['Message'])

In [51]:
response

{'Attributes': {'info': {'rating': Decimal('5.5'),
   'actors': ['Larry', 'Moe', 'Curly'],
   'plot': 'Everything happens all at once.'}},
 'ResponseMetadata': {'RequestId': 'f92ce558-7075-48e7-a6e4-8a4bdb6e9bef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2682384945',
   'x-amzn-requestid': 'f92ce558-7075-48e7-a6e4-8a4bdb6e9bef',
   'content-length': '156',
   'server': 'Jetty(8.1.12.v20130726)'},
  'RetryAttempts': 0}}

In [20]:
title = "The Big New Movie"
year = 2015

try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    
item

{'title': 'The Big New Movie',
 'year': Decimal('2015'),
 'info': {'rating': Decimal('5.5'),
  'actors': ['Larry', 'Moe', 'Curly'],
  'plot': 'Everything happens all at once.'}}

#####  Update an Item conditionally

In [21]:
actor_count = 2
try:
    response = table.update_item(
        Key={
            'year': year,
            'title': title
        },
        UpdateExpression="remove info.actors[0]",
        ConditionExpression="size(info.actors) > :num",
        ExpressionAttributeValues={':num': actor_count},
        ReturnValues="UPDATED_NEW"
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise

In [61]:
# update successful if actor_count < 3: "Larry" is removed
# The conditional request failed if actor_count >= 3

In [23]:
response

{'Attributes': {'info': {'rating': Decimal('5.5'),
   'actors': ['Moe', 'Curly'],
   'plot': 'Everything happens all at once.'}},
 'ResponseMetadata': {'RequestId': 'a68beaf6-c8df-4e24-ae79-2078add578b9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '1320246852',
   'x-amzn-requestid': 'a68beaf6-c8df-4e24-ae79-2078add578b9',
   'content-length': '142',
   'server': 'Jetty(8.1.12.v20130726)'},
  'RetryAttempts': 0}}

#### Step 3.6: Delete an Item

In [64]:
year = 2013
title = "Prisoners"

In [65]:
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    
item

{'title': 'Prisoners',
 'year': Decimal('2013'),
 'info': {'actors': ['Hugh Jackman', 'Jake Gyllenhaal', 'Viola Davis'],
  'release_date': '2013-08-30T00:00:00Z',
  'plot': "When Keller Dover's daughter and her friend go missing, he takes matters into his own hands as the police pursue multiple leads and the pressure mounts. But just how far will this desperate father go to protect his family?",
  'genres': ['Crime', 'Drama', 'Thriller'],
  'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTg0NTIzMjQ1NV5BMl5BanBnXkFtZTcwNDc3MzM5OQ@@._V1_SX400_.jpg',
  'directors': ['Denis Villeneuve'],
  'rating': Decimal('8.2'),
  'rank': Decimal('3'),
  'running_time_secs': Decimal('9180')}}

In [68]:
try:
    response = table.delete_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        item = response['Item']

In [69]:
response

{'ConsumedCapacity': {'TableName': 'Movies', 'CapacityUnits': 1.0},
 'ResponseMetadata': {'RequestId': '1e393259-ea64-4015-a245-37f509b59b37',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2336287325',
   'x-amzn-requestid': '1e393259-ea64-4015-a245-37f509b59b37',
   'content-length': '63',
   'server': 'Jetty(8.1.12.v20130726)'},
  'RetryAttempts': 0}}

In [70]:
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    
item

KeyError: 'Item'

##### Delete an Item conditionally

In [25]:
year = 2013
title = "The Hunger Games: Catching Fire"
rank = 5

In [26]:
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    
item

{'title': 'The Hunger Games: Catching Fire',
 'year': Decimal('2013'),
 'info': {'actors': ['Jennifer Lawrence', 'Josh Hutcherson', 'Liam Hemsworth'],
  'release_date': '2013-11-11T00:00:00Z',
  'plot': 'Katniss Everdeen and Peeta Mellark become targets of the Capitol after their victory in the 74th Hunger Games sparks a rebellion in the Districts of Panem.',
  'genres': ['Action', 'Adventure', 'Sci-Fi', 'Thriller'],
  'image_url': 'http://ia.media-imdb.com/images/M/MV5BMTAyMjQ3OTAxMzNeQTJeQWpwZ15BbWU4MDU0NzA1MzAx._V1_SX400_.jpg',
  'directors': ['Francis Lawrence'],
  'rank': Decimal('4'),
  'running_time_secs': Decimal('8760')}}

In [27]:
try:
    response = table.delete_item(
        Key={
            'year': year,
            'title': title
        },
        ConditionExpression="info.#rnk <= :val",
        ExpressionAttributeValues={
            ":val": decimal.Decimal(rank)
        },
        ExpressionAttributeNames={
            "#rnk": "rank"
        }
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise

In [28]:
try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    
item

KeyError: 'Item'

### [Step 4: Query and Scan the Data](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.04.html)


- [Working with Queries](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Query.html)
    - begins_with
    - between
    - eq
    - gt 
    - gte
    - lt
    - lte
    - mro

#### Query - All Movies Released in a Year

In [48]:
year = 2009
table = dynamodb.Table('Movies')
response = table.query(
    # KeyConditionExpression=Key('year').eq(year) & Key('title').begins_with('H')
    # KeyConditionExpression=Key('year').eq(year) & Key('title').gt('H')
    KeyConditionExpression=Key('year').eq(year) & Key('title').between('B', 'D')
)

In [49]:
response

{'Items': [{'title': 'Bakjwi',
   'year': Decimal('2009'),
   'info': {'actors': ['Kang-ho Song', 'Ok-bin Kim', 'Hae-suk Kim'],
    'release_date': '2009-04-30T00:00:00Z',
    'plot': 'Through a failed medical experiment, a priest is stricken with vampirism and is forced to abandon his ascetic ways.',
    'genres': ['Drama', 'Horror', 'Thriller'],
    'image_url': 'http://ia.media-imdb.com/images/M/MV5BNDgwODY0MjM3OV5BMl5BanBnXkFtZTcwNzk3MTY2Mg@@._V1_SX400_.jpg',
    'directors': ['Chan-wook Park'],
    'rating': Decimal('7.1'),
    'rank': Decimal('3837'),
    'running_time_secs': Decimal('7980')}},
  {'title': 'Bandslam',
   'year': Decimal('2009'),
   'info': {'actors': ['Aly Michalka', 'Vanessa Hudgens', 'Gaelan Connell'],
    'release_date': '2009-08-06T00:00:00Z',
    'plot': 'A new kid in town assembles a fledgling rock band -- together, they achieve their dreams and compete against the best in the biggest event of the year, a battle of the bands.',
    'genres': ['Comedy', 'Dra

In [50]:
len(response['Items'])

27

In [51]:
titles = [i['title'] for i in response['Items']]
titles

['Bakjwi',
 'Bandslam',
 'Banlieue 13 - Ultimatum',
 'Bitch Slap',
 'Black Dynamite',
 'Blood and Bone',
 'Bride Wars',
 'Bright Star',
 "Brooklyn's Finest",
 'Brothers',
 'Bruno',
 'Carriers',
 'Case 39',
 'Celda 211',
 'Cherrybomb',
 "Cirque du Freak: The Vampire's Assistant",
 'City Island',
 'Cloudy with a Chance of Meatballs',
 'Coco avant Chanel',
 'Confessions of a Shopaholic',
 'Coraline',
 'Couples Retreat',
 'Cracks',
 'Crank: High Voltage',
 'Crazy Heart',
 'Creation',
 'Crossing Over']

#### Query - All Movies Released in a Year with Certain Titles

In [52]:
year = 1992
title_range = ('B', 'E')

# Expression attribute names can only reference items in the projection expression.
# year is a reserved word, use ExpressionAttributeNames to remap it
response = table.query(
    ProjectionExpression="#yr, title, info.genres, info.actors[0]",
    ExpressionAttributeNames={"#yr": "year"},
    KeyConditionExpression=
        Key('year').eq(year) & Key('title').between(title_range[0], title_range[1])
)

In [84]:
response['Items']

[{'title': 'A Few Good Men',
  'year': Decimal('1992'),
  'info': {'actors': ['Tom Cruise'],
   'genres': ['Crime', 'Drama', 'Mystery', 'Thriller']}},
 {'title': 'A League of Their Own',
  'year': Decimal('1992'),
  'info': {'actors': ['Tom Hanks'], 'genres': ['Comedy', 'Drama', 'Sport']}},
 {'title': 'A River Runs Through It',
  'year': Decimal('1992'),
  'info': {'actors': ['Craig Sheffer'], 'genres': ['Drama']}},
 {'title': 'Aladdin',
  'year': Decimal('1992'),
  'info': {'actors': ['Scott Weinger'],
   'genres': ['Animation',
    'Adventure',
    'Comedy',
    'Family',
    'Fantasy',
    'Musical',
    'Romance']}},
 {'title': 'Alien 3',
  'year': Decimal('1992'),
  'info': {'actors': ['Sigourney Weaver'],
   'genres': ['Action', 'Sci-Fi', 'Thriller']}},
 {'title': 'Army of Darkness',
  'year': Decimal('1992'),
  'info': {'actors': ['Bruce Campbell'],
   'genres': ['Comedy', 'Fantasy', 'Horror']}},
 {'title': 'Batman Returns',
  'year': Decimal('1992'),
  'info': {'actors': ['Mich

In [54]:
titles = [i['title'] for i in response['Items']]
titles

['Batman Returns',
 'Beethoven',
 'Bitter Moon',
 'Boomerang',
 'Braindead',
 'Buffy the Vampire Slayer',
 'Candyman',
 'Captain Ron',
 'Chaplin',
 'Como agua para chocolate',
 'Damage',
 'Death Becomes Her']

#### Scan with pagination

In [55]:
year_range = (1950, 1955)
scan_kwargs = {
    'FilterExpression': Key('year').between(*year_range),
    'ProjectionExpression': "#yr, title, info.rating",
    'ExpressionAttributeNames': {"#yr": "year"}
}

In [56]:
done = False
start_key = None
movies = []
ipage = 0
while not done:
    ipage += 1
    print(f"ipage = {ipage}")
    if start_key:
        scan_kwargs['ExclusiveStartKey'] = start_key
    response = table.scan(**scan_kwargs)
    movies.extend(response.get('Items', []))
    start_key = response.get('LastEvaluatedKey', None)
    done = start_key is None

ipage = 1
ipage = 2


In [57]:
len(movies), movies

(38,
 [{'title': 'High Noon',
   'year': Decimal('1952'),
   'info': {'rating': Decimal('8.2')}},
  {'title': "Singin' in the Rain",
   'year': Decimal('1952'),
   'info': {'rating': Decimal('8.4')}},
  {'title': 'The Member of the Wedding',
   'year': Decimal('1952'),
   'info': {'rating': Decimal('6.8')}},
  {'title': 'The Quiet Man',
   'year': Decimal('1952'),
   'info': {'rating': Decimal('7.8')}},
  {'title': '20000 Leagues Under the Sea',
   'year': Decimal('1954'),
   'info': {'rating': Decimal('7.1')}},
  {'title': 'Black Widow',
   'year': Decimal('1954'),
   'info': {'rating': Decimal('6.7')}},
  {'title': 'Dial M for Murder',
   'year': Decimal('1954'),
   'info': {'rating': Decimal('8.1')}},
  {'title': 'On the Waterfront',
   'year': Decimal('1954'),
   'info': {'rating': Decimal('8.3')}},
  {'title': 'Rear Window',
   'year': Decimal('1954'),
   'info': {'rating': Decimal('8.6')}},
  {'title': 'Seven Brides for Seven Brothers',
   'year': Decimal('1954'),
   'info': {'ra